In [ ]:
"""
Get articles/texts, and filter only sentences with white-listed vocab OR named entities.

Contents:
-load packages
-create helper functions
-data intialization from csv file of words &
 extend vocab set
-build data structure to house info moving forward
-write to .json formats
-read from existing .json formats
-get white-listed of words and inflections
-scrape journale en francais facile
"""

In [1]:
import bs4, requests, sys, codecs, urllib.request, re
from bs4 import SoupStrainer
from bs4.element import Comment
import random
import string
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import mlconjug3
import sklearn
import goslate
from nltk.tokenize import word_tokenize, sent_tokenize
import pprint
pp = pprint.PrettyPrinter()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)
import mitosheet
import translators as ts

Using state Washington server backend.


In [2]:
def add_noun(add_word):

    if add_word[len(add_word)-1] != 's' and add_word[len(add_word)-1] != 'x':

        out_word = add_word + 's'

        return out_word
    else:
        return add_word

In [3]:
##METHODS

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = bs4.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



user_agent = 'Mozilla/5.0 (Windows NT 6.3; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers={'User-Agent':user_agent,}


In [4]:
def get_known(unknown_list):
    
    count_got = 0
    known_list = []
    count_checked = 0
    for word in unknown_list:
        count_checked += 1
        decision = str(input(word+"\nKnown =k"))
        if decision =='k':
            known_list.append(word)
            count_got +=1
            print("got "+str(count_got))
        elif decision=='q':
            break
        print(str(len(unknown_list)-count_checked)+' remaining')
    return known_list

In [19]:
def review_out(out_dict,prev_learned,eb_unk_in,eb_unk_m_in,examples,counters):
    """
    out_dict = out
    prev_learned = eb_learned
    eb_unk_in =  eb_unk
    eb_unk_m_in = eb_unk_morph
    examples = eb_unk_examples
    
    decisions:
    k = word known, remove from eb_unk and eb_unk_morph
        and add to eb_learned
        
    m = remove morph from eb_unk and eb_unk_morph 
    
    a = add example to eb_unk_examples
    q = quit for day
    """
    print(str(len(out_dict.keys()))+" total")
    counter = 0
    decision = ''
    
    for word in out_dict.keys():
        if decision !='q':

            counter += 1
            print(str(counter)+" out of "+str(len(out_dict.keys())))
            print("Lemma: "+word)
            print('')
            print("Morph: "+str(out_dict[word]['morph']))



            if len(examples[word])>0:
                print("Previous examples: ")
                for sent in examples[word]:
                    print(" - "+sent)

            print("New examples: ")

            i = 0
            decision = ''
            while (i < len(out_dict[word]['sents'])) and (decision!='s'):

                sent = out_dict[word]['sents'][i]
                if (sent not in examples[word]) and (sent not in counters[word]):
                    print(sent)
                    print('')
                    decision = str(input("options : k,m,a,q,s,c"))
                    if decision =='k':
                        try:
                            examples[word].append(sent) #by default adds examples
                            prev_learned[word] = eb_unk_in[word].copy()
                            del eb_unk_in[word] #remove lemma

                            i = 0
                            while i < len(eb_unk_m_in):

                                if eb_unk_m_in[list(eb_unk_m_in)[i]] == word:
                                   del eb_unk_m_in[list(eb_unk_m_in)[i]]
                                else:
                                    i +=1
                        except:
                            pass

                    if decision =='m':
                        morph = str(input("which morph:"))
                        try:
                            del eb_unk_m_in[morph]
                            eb_unk_in[word].remove(morph) 
                        except:
                            pass
                    if decision =='a':
                        try:
                            examples[word].append(sent)
                        except:
                            pass
                    if decision=='c':
                        counters[word].append(sent)

                i += 1
    

In [6]:
def how_to_add(word):
    """
    p = plain, add without conjurgating or changing
    c = conjugate as normal verb
    x = noun that adds x when plural
    s = noun that add s when plural
    iv = inner verb. There is a verb in the expression that should be 
        conjurgated
    """
    extras_morphs = []
    extras_morphs.append(word)
    
    
    gs = goslate.Goslate()

    default_conjugator = mlconjug3.Conjugator(language='fr')

    decision = str(input(word+"\nc\nx\ns\niv"))
    
    if decision == 'c':

        try:
            temp = []
            test_verb = default_conjugator.conjugate(word)
            all_conjugated_forms = test_verb.iterate()
            for item in all_conjugated_forms:
                if item not in temp:
                    temp.append(item[len(item)-1])
            extras_morphs.extend(list(set(temp)))
        except:
            pass
    elif decision == 'x':
        extras_morphs.append(lemma+'x')
    elif decision == 's':
        extras_morphs.append(lemma+'s')
    elif decision == 'iv':
        morph = str(input("which morph:"))
        

        temp = []
        test_verb = default_conjugator.conjugate(morph)
        all_conjugated_forms = test_verb.iterate()
        for item in all_conjugated_forms:
            if item not in temp:
                temp.append(item[len(item)-1])

        for variant in list(set(temp)):
            if variant is not None:
                new_word = word.replace(morph,variant)
                extras_morphs.append(new_word)

    return extras_morphs

In [7]:
"""
For eb_unk words: search through all articles, return 
sentences containing any words for unknown vocabulary. 

return dict object, key is lemma, then values are morphs found
and array of examples. 
"""

def find_unks(webpages,eb_unk_mo,prev_examples, prev_counter_examples):
    
    output = {}
    for webpage in webpages:
        try:

            request=urllib.request.Request(webpage,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read()
            contents = text_from_html(data)

            contents_array = sent_tokenize(contents)

            for line in contents_array:
                tokenized = word_tokenize(line, language='french')
                for word in tokenized:
                    if word.lower() in eb_unk_mo:
                        if not bool(re.search('À lire aussi',line)):

                            if word.lower() in prev_examples:
                                if (line not in prev_examples[word.lower()]) and \
                                (line not in prev_counter_examples[word.lower()]):
                                    if word.lower() not in output:
                                        output[eb_unk_mo[word.lower()]] = {} #key will be lemma
                                        output[eb_unk_mo[word.lower()]]['morph'] = []
                                        output[eb_unk_mo[word.lower()]]['sents'] = []

                                    #disallow "" phrases

                                    output[eb_unk_mo[word.lower()]]['morph'].append(word.lower())
                                    output[eb_unk_mo[word.lower()]]['sents'].append(line)

        except:
            pass
    return output
    

In [8]:
def filter_text(webpage,text=None,print_word_lvl=False):


    #try:
    if text is None:
        request=urllib.request.Request(webpage,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read()
        contents = text_from_html(data)
    else:
        contents = webpage

    line_array = []
    percent_array = []
    contents_array = sent_tokenize(contents)

    disallowed_words = set()
    total_words = 0
    unknown_words = 0
    lines = []
    unknowns = []
    
    for line in contents_array:
        
        line_total = 0
        line_unks = 0
        tokenized = word_tokenize(line, language='french')

        unk_str = ""
        for word in tokenized:
            total_words +=1
            line_total +=1

            if not (bool(re.search("[0-9]", word)) or\
                    bool(re.search("[A-Z]", word)) or\
                    bool(re.search("[.,\/#!$%\^&\*;:{}=\-_`~()«»]", word)) or\
                    word.lower() in vocab['white_listed']):
                if not ((word.lower() in vocab['white_listed']) or (word.lower() in vocab['black_listed'])):
                    unk_str += '"'+word.lower()+'"'+", "

                disallowed_words.add(word.lower())
                unknown_words +=1
                line_unks +=1


        line_array.append(line)
        line_percent = (line_total-line_unks)/line_total
        percent_array.append(line_percent)

        if len(unk_str)>0:
            unk_str = unk_str[0:len(unk_str)-1]

        unknowns.append(unk_str)

    if print_word_lvl and total_words>0:
        print("word-level % known = "+str((1-(unknown_words/total_words))*100))

    return_pd = pd.DataFrame(list(zip(line_array,percent_array,unknowns)))
    return_pd.columns = ["line","line_percent","unk_words"]

    with open(path+'unknown_french_dad_list.txt',"w") as outfile:
        outfile.write(str(list(disallowed_words)))

    return return_pd
    
    #except:
    #    pass

In [9]:
path = "/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/"
source_file = "french_dad.json"

##Dad section

In [273]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13675


In [274]:
            
##Journal en francais facile:
print("Checking Journal en francais facile")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://francaisfacile.rfi.fr/fr/podcasts/journal-en-fran%c3%a7ais-facile/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))


pages = []
for link in soup.find_all('a', href=True):
    if '/fr/podcasts/le-journal-en-fran%C3%A7ais-facile/' in str(link['href']):
        #print(str(link['href']))
        str_page = 'https://savoirs.rfi.fr/'+str(link['href'])
        #str_page = 'https://savoirs.rfi.fr/en/apprendre-enseigner/langue-francaise/journal-en-francais-facile-'+str(re.search(r"[0-9]{8}.*",str(link['href'])).group(0))
        if str_page not in pages:
            pages.append(str_page)
print("open file:")
print(path+"text_input_web_dad_fr.txt")
for page in pages:
    print(page)

Checking Journal en francais facile
open file:
/Users/elyebliss/Desktop/Vocabulary/language_learning/vocab_dfs/text_input_web_dad_fr.txt
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221207-covid-la-chine-desserre-l-%C3%A9tau-la-cop15-%C3%A0-montr%C3%A9al-le-mondial-de-foot
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221206-l-ue-lutte-contre-la-d%C3%A9forestation-la-tv-russe-dojd-interdite-samuel-eto-o-frappe-un-youtubeur
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221206-vladimir-poutine-bombarde-l-ukraine-appel-%C3%A0-la-gr%C3%A8ve-en-iran-nicolas-sarkozy-devant-les-juges
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221202-cyril-ramaphosa-dans-la-tourmente-e-macron-%C3%A0-la-nouvelle-orl%C3%A9ans-kanye-west-d%C3%A9rape
https://savoirs.rfi.fr//fr/podcasts/le-journal-en-fran%C3%A7ais-facile/20221201-sida-la-lutte-continue-massacre-en-ha%C3%AFti-l-op%C3%A9ra-de-paris-recrute-

In [277]:
with open(path+"text_input_web_dad_fr.txt","r") as infile:
    textfile = infile.read()
filtered_art = filter_text(textfile,text= True)
display(filtered_art)

,line,line_percent,unk_words
0,"La vie quotidienne des Chinois devrait être plus simple car les autorités ont annoncé un allègement, un assouplissement des mesures sanitaires contre le Covid.",0.884615,"""quotidienne"", ""allègement"", ""assouplissement"","
1,Reportage à Pékin.,1.000000,
2,La guerre en Ukraine.,1.000000,
3,De nouveaux bombardements ont fait aujourd'hui au moins six morts.,1.000000,
4,Des civils qui habitaient la ville de Kourakhove.,1.000000,
...,...,...,...
88,Le quart entre les Pays-Bas et l'Argentine est un grand classique du Mondial.,0.928571,"""quart"","
89,"Le Brésil, qui brigue ici une sixième étoile, retrouve le finaliste d'il y a quatre ans, la Croatie.",0.818182,"""brigue"", ""sixième"", ""finaliste"", ""d'il"","
90,"Quant à l'équipe de France, championne du monde en titre, elle croisera le fer avec une Angleterre demi - finaliste en 2018 et des Anglais qui font partie des huit pays à avoir déjà soulevé le trophée.",0.875000,"""championne"", ""croisera"", ""finaliste"", ""soulevé"", ""trophée"","
91,C'était en 1966.,1.000000,


In [276]:
all_unks = set()

with open(path+'unknown_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [re.sub("^['|\"]|^\['|['|\"]$|'\]$|\[","",line.strip().lower()) for line in new_words] #update regex
    for word in new_words:
        all_unks.add(word)
len(all_unks)

164

In [278]:
all_unks=list(all_unks)
unk_df = pd.DataFrame(all_unks)
unk_df.columns = ['word']
unk_df['status'] = pd.Series(['' for word in all_unks])

In [279]:
mitosheet.sheet(unk_df, analysis_to_replay="id-mdbmtwtdut")

MitoWidget(analysis_data_json='{"analysisName": "id-mdbmtwtdut", "analysisToReplay": null, "code": [], "stepSu…

In [281]:
add_ints = [1, 2, 4, 5, 8, 15, 19, 20, 21, 26, 27, 33, 36, 37, 41, 42, 50, 51, 53, 56, 57, 61, 67, 71, 72, 76, 78, 83, 89, 91, 96, 106, 108, 111, 115, 126, 135, 140, 142, 144, 145, 146, 147, 148, 152, 154, 155, 157, 162, 163]

In [282]:
for i in add_ints:
    # Set a cell value in status
    unk_df.at[i, 'status'] = 'k'

In [23]:
translations = []
for i in range(0,len(unk_df)):
    try:
        translations.append(ts.google(unk_df.word.iloc[i]))
    except:
        translations.append('')
unk_df['translations']=translations

In [24]:
mitosheet.sheet(unk_df, analysis_to_replay="id-ianxevwttg")

MitoWidget(analysis_data_json='{"analysisName": "id-ianxevwttg", "analysisToReplay": null, "code": [], "stepSu…

In [283]:
add_words = list(unk_df.word[unk_df['status']=='k'])
print(len(add_words))
for word in add_words:
    vocab['white_listed'].add(word)

50


In [284]:
filtered_art = filter_text(textfile,text= True)

In [285]:
filtered_art['theme']=['' for line in filtered_art.line]
filtered_art = filtered_art[['theme','unk_words','line','line_percent']]
with open(path+'dad_choose_themes.xlsx','wb') as outfile:
    filtered_art.to_excel(outfile, encoding='utf-8',index=False)

In [286]:
remove_words = ['colère']

for word in remove_words:
    if word in vocab['white_listed']:
        vocab['white_listed'].remove(word)

In [287]:
with open(path+'dad_choose_themes.xlsx','rb') as infile:
    filtered_art = pd.read_excel(infile)

In [288]:
themes = set(filtered_art.theme[filtered_art['theme']!='NaN'])
output = pd.DataFrame(columns=['theme','line'])
for theme in themes:
    temp = filtered_art[['theme','line']][filtered_art['theme']==theme]
    output = pd.concat([output,temp])
with open(path+'dad_article.xlsx','wb') as outfile:
    output.to_excel(outfile, encoding='utf-8',index=False)

In [289]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(vocab['white_listed']))

13724


In [290]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13724


##Filter for lessons

In [264]:
with open(path+"text_input_dad_fr.txt","r") as infile:
    textfile = infile.read()
filtered_art = filter_text(textfile,text= True)
display(filtered_art)

,line,line_percent,unk_words
0,Les hommes ont regardé le chef Eskol.,1.000000,
1,–On va abandonner nos familles ?,1.000000,
2,Nos amis ?,1.000000,
3,On ne peut pas faire ça !,1.000000,
4,–a crié un Viking.,1.000000,
...,...,...,...
40,Les hommes ont ri.,1.000000,
41,Il n’y avait pas de haine entre eux.,0.909091,"""haine"","
42,Il y avait seulement de la joie.,1.000000,
43,Ils étaient prêts pour leur prochaine expédition… prêts à explorer un nouveau monde.,0.928571,"""expédition…"","


In [265]:
with open(path+'unknown_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [re.sub("^['|\"]|^\['|['|\"]$|'\]$","",line.strip().lower()) for line in new_words] #update regex
#new_words
print(len(new_words))
known_manual = get_known(new_words)

17
établir
Known =k
16 remaining
essayerait
Known =k
15 remaining
inquiétez
Known =k
14 remaining
haine
Known =k
13 remaining
libres
Known =k
12 remaining
formés
Known =k
11 remaining
–lui
Known =kk
got 1
10 remaining
désolé
Known =kk
got 2
9 remaining
ancre
Known =k
8 remaining
choix
Known =kk
got 3
7 remaining
endommagés
Known =k
6 remaining
forcer
Known =kk
got 4
5 remaining
plaisaient
Known =k
4 remaining
fonctionné
Known =k
3 remaining
expédition…
Known =kk
got 5
2 remaining
trous
Known =k
1 remaining
suivants
Known =k
0 remaining


In [266]:
print(len(known_manual))
for word in known_manual:
    vocab['white_listed'].add(word)

5


In [267]:
#known_lines_only = list(set(list(filtered_art.knowns[filtered_art.knowns != "..."])))
#known_lines_only = sorted(known_lines_only,key=len)
filtered_art = filter_text(textfile,text= True)
known_lines_only = list(filtered_art.line[filtered_art['line_percent']==1.0])
known_lines_only = sorted(known_lines_only,key=len)


In [268]:
known_lines_only

['Nos amis ?',
 '–Peu importe.',
 '–a crié un Viking.',
 'Les hommes ont ri.',
 'On va être heureux.',
 '–Oui, –a dit Thoric.',
 '–Regardez nos bateaux !',
 '–a dit un autre \nhomme.',
 'On ne peut pas faire ça !',
 'La tempête les a détruits !',
 'Je ne vais pas vous forcer.',
 '–On va abandonner nos familles ?',
 '–Vous avez raison tous les deux.',
 'Il y avait seulement de la joie.',
 'C’est pourquoi vous avez le choix.',
 'Le chef Eskol a regardé ses hommes.',
 '–C’est un endroit très intéressant.',
 'Pas avec des bateaux dans cet état !',
 'Je suis simplement un ancien Viking.',
 'Les hommes ont regardé le chef Eskol.',
 '–Je suis vraiment désolé, –a dit Eskol.',
 'Un mois plus tard, le deuxième groupe est parti.',
 'Vous vouliez seulement le bien de notre village.',
 '–Nous ne pouvons pas retourner en toute sécurité.',
 'A partir de maintenant, je ne suis plus votre chef.',
 'Si vous choisissez de rester, vous êtes les bienvenus.',
 'Un groupe a décidé de rester sur les nouvelles

In [269]:
#export knowns: [filtered_art.knowns != "..."]
#known_list = list(filtered_art.knowns)
with open(path+"known_lines_dad_fr.txt","w") as outfile:
    for line in known_lines_only:
        outfile.write(line+"\n")

In [270]:
#write to .json formats
df = vocab
df['white_listed'] = list(df['white_listed'])
df['black_listed'] = list(df['black_listed'])
with open(path+source_file, "w") as outfile:
    json.dump(df,outfile)
print(len(vocab['white_listed']))

13675


In [271]:
#read from existing .json formats
with open(path+source_file, "r") as path_in:
    vocab = json.loads(path_in.read())
vocab['white_listed'] = set(vocab['white_listed'])
vocab['black_listed'] = set(vocab['black_listed'])
print(len(vocab['white_listed']))

13675


##EB section

In [10]:
#read eb unknown files from .json formats
"""
Remaining vocab size = 2177
Total learned = 190
"""
with open(path+'eb_unk.json', "r") as infile:
    eb_unk = json.loads(infile.read())
print("Remaining vocab size = "+str(len(eb_unk)))

with open(path+'eb_unk_mo.json', "r") as infile:
    eb_unk_mo = json.loads(infile.read())

with open(path+'eb_learned.json', "r") as infile:
    eb_learned= json.loads(infile.read())
print("Total learned = "+str(len(eb_learned)))

with open(path+'eb_unk_examples.json', "r") as infile:
    eb_unk_examples= json.loads(infile.read())

with open(path+'eb_unk_counter_examples.json', "r") as infile:
    eb_unk_counter_examples= json.loads(infile.read())

Remaining vocab size = 2177
Total learned = 190


In [233]:
"""
space = plain, add without conjurgating or changing
c = conjugate as normal verb
x = noun that adds x when plural
s = noun that add s when plural
iv = inner verb. There is a verb in the expression that should be 
    conjurgated
"""
with open(path+"fr_eb_extra.txt","r") as infile:
    extras_lemmas = infile.read().split('\n')

for lemma in extras_lemmas:
    extras_morphs = how_to_add(lemma)
    
    eb_unk[lemma] = extras_morphs
    for morph in extras_morphs:        
        if morph not in eb_unk_mo:
            eb_unk_mo[morph] = lemma

print("Remaining vocab size = "+str(len(eb_unk)))

#need to add above to examples and counter examples
for lemma in eb_unk:
    if lemma not in eb_unk_examples:
        eb_unk_examples[lemma] = []
    if lemma not in eb_unk_counter_examples:
        eb_unk_counter_examples[lemma]= []


c
x
s
iv
Remaining vocab size = 2200


Get larger list of pages to search

In [11]:
websites = set()
with open(path+'viewed_websites_fr.txt',"r") as infile:
    for line in infile.read().split('\n'):
        websites.add(line)
print(len(websites))

830


In [12]:
pages = []

In [13]:
##France24
print("Checking France24")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.france24.com/fr/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))



for link in soup.find_all('a', href=True):
    str_ver = str(link['href'])
    
    #get regex such that links start with /fr/ and don't end
    #with /
    if bool(re.search('^\/fr\/.*',str_ver)) and (str_ver[len(str_ver)-1]!='/'):
        pages.append('https://www.france24.com'+str_ver)

Checking France24


In [14]:
##20minutes
print("Checking 20minutes")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.20minutes.fr/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    str_ver = str(link['href'])
    
    final_segment = str_ver.split('/')[len(str_ver.split('/'))-1]
    #articles have https:// at the beginning and a date in the last segment
    if ('https://www.20minutes.fr' in str_ver)\
    and bool(re.search('[0-9]',final_segment)):
        pages.append(str_ver)

Checking 20minutes


In [15]:
##TV5Monde
print("Checking TV5Monde")
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#request=urllib.request.Request('https://savoirs.rfi.fr/en/apprendre-enseigner/langue-fran%C3%A7aise/journal-en-francais-facile',None,headers)
request=urllib.request.Request('https://www.tv5monde.com/',None,headers)

resp = urllib.request.urlopen(request)
soup = bs4.BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

for link in soup.find_all('a', href=True):
    if '/info/' in str(link['href']):
        #print(str(link['href']))
        #str_page = 'https://savoirs.rfi.fr/'+str(link['href'])
        #str_page = 'https://savoirs.rfi.fr/en/apprendre-enseigner/langue-francaise/journal-en-francais-facile-'+str(re.search(r"[0-9]{8}.*",str(link['href'])).group(0))
        if str(link['href']) not in pages:
            pages.append(str(link['href']))


Checking TV5Monde


In [16]:
pages = list(set(pages))

#add in set_difference line here
print("new = "+str(len(set(pages).difference(websites))))
pages = list(set(pages).difference(websites))

new = 94


In [17]:
out = find_unks(pages,eb_unk_mo,eb_unk_examples,eb_unk_counter_examples)

In [20]:
review_out(out,eb_learned,eb_unk,eb_unk_mo,eb_unk_examples,eb_unk_counter_examples)

87 total
1 out of 87
Lemma: calvaire

Morph: ['calvaire', 'calvaire', 'calvaire', 'calvaire', 'calvaire', 'calvaire']
Previous examples: 
 - Mais on est aussi en colère par rapport à l'instrumentalisation de toute cette question et par rapport au calvaire qu'on a pu vivre.
 - Malgré le calvaire qui est le sien depuis son arrivée au Qatar , Paul impressionne par la force mentale qu’il dégage.
 - (Re)voir - Canada : fin du calvaire pour les deux Canadiens retenus en Chine (25.9.2021) ​   Ingérence de la Chine dans les élections canadiennes de 2019 ?
New examples: 
Fake Off    By the Web    High-Tech    20 Mint    Vidéos    Podcasts    En Images    Guide d'achat    Le direct    Jeux    Services    Journal numérique    Minutes Maison      Menu complet        Retour à l’accueil   Monde  Moscou admet être vulnérable en Crimée au 288e jour de guerre en Ukraine      Partager sur Messenger       Partager sur Facebook       Partager sur Twitter       Partager sur Flipboard       Partager sur Lin

options : k,m,a,q,s,cs
9 out of 87
Lemma: rédiger

Morph: ['rédiger', 'rédiger']
Previous examples: 
 - Une fois n’est pas coutume, cette année ce n’est pas Sylvie Tellier, qui s’apprête à quitter le comité Miss France , mais bien le producteur artistique de la finale du concours Frédéric Gilbert qui a été chargé de rédiger ce quiz soumis aux participantes.
 - Ces ordinateurs peuvent naviguer sur Internet, lire un film ou de la musique en streaming, échanger des emails ou rédiger un document.
 - Vous pourrez utiliser les outils de Google pour rédiger des textes, des présentations ou des tableaux de calcul.
New examples: 
Les directives anticipées, que tout un chacun peut décider de rédiger, sont un document écrit dans lequel une personne dit à quel point elle souhaite être maintenue en vie, au cas où elle se trouverait un jour dans l'incapacité d'exprimer son choix.

options : k,m,a,q,s,ca
« Elle a permis de répondre à la demande de pouvoir mourir dans la dignité, en ouvrant de nouveau

options : k,m,a,q,s,cs
12 out of 87
Lemma: riposter

Morph: ['riposte']
Previous examples: 
 - Selon l'armée israélienne, des dizaines de Palestiniens ont brûlé des pneus et lancé des pierres en direction des soldats qui ont « riposté en ouvrant le feu ».
New examples: 
"Au cours de cette opération, des suspects ont lancé des engins explosifs et ciblé à balles réelles des soldats qui ont riposte à balles réelles", a indiqué l'armée dans un communiqué.

options : k,m,a,q,s,ca
13 out of 87
Lemma: sceller

Morph: ['sceller', 'sceller', 'sceller']
Previous examples: 
 - >> L'Uruguay corrige le Ghana mais les deux équipes sont éliminées Premier de son groupe, le Portugal de Cristiano Ronaldo a de grandes chances d'éviter en huitièmes le Brésil, qui tente dans la soirée de sceller sa première place du groupe H contre le Cameroun.
New examples: 
Téléchargez l'application France 24          Territoires palestiniens   Cisjordanie   Israël   Conflit israélo-palestinien       Contenus liés       

In [ ]:
#could remove junk examples that include "À lire aussi" -> done
#could remove junk examples that include "Fake Off" -> done
# make it possible to skip until the end without reviewing all examples
# show list of unique words and option to go to them

In [21]:
for word in pages:
    websites.add(word)
with open(path+'viewed_websites_fr.txt',"w") as outfile:
    for line in websites:
        outfile.write(line+'\n')      

In [22]:
#write to eb unknown files to .json formats
"""
Remaining vocab size = 2177
Total learned = 190
"""
with open(path+'eb_unk.json', "w") as outfile:
    json.dump(eb_unk,outfile)
print("Remaining vocab size = "+str(len(eb_unk)))

with open(path+'eb_unk_mo.json', "w") as outfile:
    json.dump(eb_unk_mo,outfile)

with open(path+'eb_learned.json', "w") as outfile:
    json.dump(eb_learned,outfile)
print("Total learned = "+str(len(eb_learned)))

with open(path+'eb_unk_examples.json', "w") as outfile:
    json.dump(eb_unk_examples,outfile)

with open(path+'eb_unk_counter_examples.json', "w") as outfile:
    json.dump(eb_unk_counter_examples,outfile)

Remaining vocab size = 2177
Total learned = 190


### Code no longer in use

In [177]:
def return_line_percents(webpage):
    request=urllib.request.Request(webpage,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read()
    contents = text_from_html(data)

    known_array = []
    unk_array = []
    contents_array = sent_tokenize(contents)

    lines = []

    for line in contents_array:
        tokenized = word_tokenize(line, language='french')

        line_total = 0
        line_unks = 0
        for word in tokenized:

            line_total +=1

            if not (bool(re.search("[0-9]", word)) or\
                    bool(re.search("[A-Z]", word)) or\
                    bool(re.search("[.,\/#!$%\^&\*;:{}=\-_`~()«»]", word)) or\
                    word.lower() in vocab['white_listed']):

                    line_unks +=1

            lines.append(100*(line_total-line_unks)/line_total)
    return(lines)

In [ ]:

if (start is not None) and (stop is not None):
    contents_array=contents_array[max(start,0):min(stop,len(contents_array))]

with open(path+'unknown_french_dad_list.txt',"w") as outfile:
    outfile.write(str(list(disallowed_words)))


In [300]:
##add in extras
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/fr_eb_extra.txt","r") as infile:
    extras_lemmas = infile.read().split('\n')

gs = goslate.Goslate()

default_conjugator = mlconjug3.Conjugator(language='fr')


for lemma in extras_lemmas:
    
    decision = how_to_add(lemma)
    
    if lemma not in eb_unk:
        eb_unk[lemma] = []
        
    extras_morphs = []
    extras_morphs.append(lemma)
    try:
        #if it's an infinitive, add all conjurgations
        test_verb = default_conjugator.conjugate(lemma)
        all_conjugated_forms = test_verb.iterate()
        for item in all_conjugated_forms:
            if item not in extras_morphs:
                extras_morphs.append(item[len(item)-1])
        extras_morphs = list(set(extras_morphs))
    except:
        
        #if not an expression:
        if len(lemma.split(' '))==1:
            #add plural nouns
            if lemma[len(lemma)-1]=='u':
                extras_morphs.append(lemma+'x')
            else:
                extras_morphs.append(lemma+'s')

    eb_unk[lemma] = extras_morphs
    for morph in extras_morphs:        
        if morph not in eb_unk_mo:
            eb_unk_mo[morph] = lemma

print("Remaining vocab size = "+str(len(eb_unk)))

Remaining vocab size = 2327


In [8]:
#Upload from download
with open(path+'known_french_dad_list.txt',"r") as input_file:
    new_words = input_file.read()
    new_words = re.sub('\n',"",new_words)
    new_words = new_words.split(',')
    new_words = [line.replace('"',"").strip().lower() for line in new_words]
    print("daily catch in word count:")
    print(len(new_words))
    
    for line in new_words:   
        vocab['white_listed'].add(line.replace('"',"").strip().lower())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/known_french_dad_list.txt'

In [117]:
"""
to do:

"""

In [49]:
##INPUT-OUTPUT
#vocab list:
with open("/Users/elyebliss/Desktop/Vocabulary/vocab_dfs/dad_whitelisted.csv","r") as infile:
    whitelisted_lemmas = infile.read()


##VARIABLES
vocab_all = set()
gs = goslate.Goslate()

default_conjugator = mlconjug3.Conjugator(language='fr')


for line in whitelisted_lemmas.split('\n'):
    if len(line) > 0:
        #print(line)
        vocab = line.lower().strip()
        vocab_all.add(vocab)
        try:
            #if it's an infinitive, add all conjurgations
            test_verb = default_conjugator.conjugate(vocab)
            all_conjugated_forms = test_verb.iterate()
            for item in all_conjugated_forms:
                
                vocab_all.add(item[len(item)-1])
                
            
        except:
            #might be a noun, add plural
            vocab_all.add(add_noun(vocab))
#len(whitelisted_lemmas.split('\n'))            
#pp.pprint(vocab_all)

In [83]:
vocab = {}
vocab['white_listed'] = list(vocab_all)
vocab['black_listed'] = []


In [27]:
#used for creating eb data

with open(path+'eb_unk_lemmas.csv','r') as infile:
    eb_unk_lemmas = pd.read_csv(infile)
with open(path+'eb_unk_morph.csv','r') as infile:
    eb_unk_morph = pd.read_csv(infile)
    
eb_unk = {}
for word in eb_unk_lemmas.lemme:
    eb_unk[word] = []

    for index, row in eb_unk_morph.iterrows():
        if row.lemme == word:
            eb_unk[word].append(row.ortho)
eb_unk_mo = {}
for lemma in eb_unk.keys():
    for morph in eb_unk[lemma]:
        eb_unk_mo[morph] = lemma

In [147]:
eb_learned = {}
eb_unk_examples = {}
for word in eb_unk.keys():
    eb_unk_examples[word] = []

In [130]:
#restore to backups
eb_learned = eb_learned_backup
eb_unk = eb_unk_backup
eb_unk_mo = eb_unk_mo_backup
eb_unk_examples = eb_unk_examples